# Propose centroid replacing the masters with BERT siamese

In [1]:
# import tensorflow as tf
import keras

Using TensorFlow backend.


In [2]:
# Optimize the use of GPUs
# https://datascience.stackexchange.com/questions/23895/multi-gpu-in-keras
# https://keras.io/getting-started/faq/#how-can-i-run-a-keras-model-on-multiple-gpus
# https://stackoverflow.com/questions/56316451/how-to-use-specific-gpus-in-keras-for-multi-gpu-training

In [3]:
from __future__ import print_function, division

In [4]:
import re
import numpy as np
import pandas as pd

import os
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import sys
from annoy import AnnoyIndex
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
    
%matplotlib inline

In [5]:
from keras.layers import Conv1D, Input, Add, Activation, Dropout, Embedding, MaxPooling1D, \
    GlobalMaxPool1D, Flatten, Dense, Concatenate, BatchNormalization
from keras.models import Sequential, Model
from keras.regularizers import l2
from keras.initializers import TruncatedNormal
from keras import optimizers

In [6]:
from methods.baseline import Baseline
from methods.experiments import Experiment
from methods.evaluation import Evaluation
from methods.retrieval import Retrieval

## Auxiliary methods

## Configurações Globais

In [7]:
MAX_SEQUENCE_LENGTH_T = 20 # 100
MAX_SEQUENCE_LENGTH_D = 20 # 500
EMBEDDING_DIM = 300
MAX_NB_WORDS = 20000
'''
    Configuration
'''
epochs = 1000
freeze_train = .1 # 10% with freeze weights
best_loss = 1
best_epoch = 0
verbose = 0
loss = 1

### Parse bugs preproprecessed

In [8]:
# Domain to use
DOMAIN = 'eclipse'
METHOD = 'deepQL_weights_{}'.format(epochs)
PREPROCESSING = 'bert'
TOKEN = 'bert'
# Dataset paths
DIR = 'data/processed/{}/{}'.format(DOMAIN, PREPROCESSING)
DIR_PAIRS = 'data/normalized/{}'.format(DOMAIN)
DATASET = os.path.join('data/normalized/{}'.format(DOMAIN), '{}.csv'.format(DOMAIN))
# Glove embeddings
GLOVE_DIR='data/embed'
# Save model
SAVE_PATH = '{}_preprocessing_{}_feature@number_of_epochs@epochs_64batch({})'.format(PREPROCESSING, METHOD, DOMAIN)
SAVE_PATH_FEATURE = '{}_preprocessing_{}_feature_@number_of_epochs@epochs_64batch({})'.format(PREPROCESSING, METHOD, DOMAIN)

# Extract CORPUs
EXTRACT_CORPUS = False

In [10]:
import os

pretrained_path = 'uncased_L-12_H-768_A-12'
config_path = os.path.join(pretrained_path, 'bert_config.json')
model_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

In [11]:
from keras_bert import load_vocabulary

token_dict = load_vocabulary(vocab_path)

In [12]:
"Total vocabulary: {}".format(len(token_dict))

'Total vocabulary: 30522'

In [13]:
baseline = Baseline(DOMAIN, DIR, DATASET, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D,
                   token_dict['[CLS]'], token_dict['[SEP]'])
evaluation = Evaluation(verbose=0)
retrieval = Retrieval()
experiment = Experiment(baseline, evaluation)

In [14]:
experiment.set_retrieval(retrieval, baseline, DOMAIN)

Creating the buckets...


#### Loading bug ids in memory

In [15]:
experiment.load_ids()
len(baseline.bug_ids)

Reading bug ids


361006

#### Dicionário de títulos e descrições

In [16]:
%%time

experiment.load_bugs(TOKEN)
len(baseline.sentence_dict)


CPU times: user 30.3 s, sys: 3.2 s, total: 33.5 s
Wall time: 33.3 s


#### Hashing bugs by buckets

In [17]:
issues_by_buckets = experiment.get_buckets_for_bugs()

#### Prepare the train and test

In [18]:
%%time

experiment.prepare_dataset(issues_by_buckets, path_train='train_chronological', path_test='test_chronological')
# Read and create the test queries duplicates
retrieval.create_queries()

Reading train data
Reading bug ids
CPU times: user 2min 36s, sys: 23 ms, total: 2min 36s
Wall time: 2min 36s


In [ ]:
baseline.train_data[:10]

#### Recovery bug ids from train

In [19]:
bug_train_ids = experiment.get_train_ids(baseline.train_data)

### Export the corpus train

In [ ]:
if EXTRACT_CORPUS:
    corpus = []
    export_file = open(os.path.join(DIR, 'corpus_train.txt'), 'w')
    for bug_id in tqdm(baseline.bug_set):
        bug = baseline.bug_set[bug_id]
        title = bug['title']
        desc = bug['description']
        export_file.write("{}\n{}\n".format(title, desc))
    export_file.close()

In [ ]:
idx = np.random.choice(baseline.bug_ids, 1)[0]
baseline.bug_set[idx]

In [ ]:
"Train ", len(baseline.dup_sets_train)

### Train ids

In [ ]:
import random

# data - path
# batch_size - 128
# n_neg - 1
def batch_iterator(self, retrieval, model, data, dup_sets, bug_ids, 
                   batch_size, n_neg, issues_by_buckets, TRIPLET_HARD=False, FLOATING_PADDING=False):
    # global train_data
    # global self.dup_sets
    # global self.bug_ids
    # global self.bug_set

    random.shuffle(data)

    batch_features = {'title' : [], 'desc' : [], 'info' : []}
    n_train = len(data)

    batch_triplets, batch_bugs_anchor, batch_bugs_pos, batch_bugs_neg, batch_bugs = [], [], [], [], []

    all_bugs = list(issues_by_buckets.keys())
    buckets = retrieval.buckets

    for offset in range(batch_size):
        anchor, pos = data[offset][0], data[offset][1]
        batch_bugs_anchor.append(anchor)
        batch_bugs_pos.append(pos)
        batch_bugs.append(anchor)
        batch_bugs.append(pos)
        #batch_bugs += dup_sets[anchor]

    for anchor, pos in zip(batch_bugs_anchor, batch_bugs_pos):
        while True:
            neg = self.get_neg_bug(anchor, buckets[issues_by_buckets[anchor]], issues_by_buckets, all_bugs)
            bug_anchor = self.bug_set[anchor]
            bug_pos = self.bug_set[pos]
            if neg not in self.bug_set:
                continue
            batch_bugs.append(neg)
            batch_bugs_neg.append(neg)
            bug_neg = self.bug_set[neg]
            break
        
        # triplet bug and master
        batch_triplets.append([anchor, pos, neg])
    
    random.shuffle(batch_bugs)
    title_ids = np.full((len(batch_bugs), MAX_SEQUENCE_LENGTH_T), 0)
    description_ids = np.full((len(batch_bugs), MAX_SEQUENCE_LENGTH_D), 0)
    for i, bug_id in enumerate(batch_bugs):
        bug = self.bug_set[bug_id]
        self.read_batch_bugs(batch_features, bug, index=i, title_ids=title_ids, description_ids=description_ids)

    batch_features['title'] = { 'token' : np.array(batch_features['title']), 'segment' : title_ids }
    batch_features['desc'] = { 'token' : np.array(batch_features['desc']), 'segment' : description_ids }
    batch_features['info'] = np.array(batch_features['info'])
    
    sim = np.asarray([issues_by_buckets[bug_id] for bug_id in batch_bugs])

    input_sample = {}

    input_sample = { 'title' : batch_features['title'], 
                        'description' : batch_features['desc'], 
                            'info' : batch_features['info'] }

    return batch_triplets, input_sample, sim #sim

In [24]:
# %%time

batch_size = 64
batch_size_test = 128

# we want a constant validation group to have a frame of reference for model performance
batch_triplets_valid, valid_input_sample, valid_sim = batch_iterator(baseline, retrieval, None, 
                                                                                      baseline.train_data, 
                                                                                      baseline.dup_sets_train,
                                                                                      bug_train_ids,
                                                                                      batch_size_test, 1,
                                                                                      issues_by_buckets)

validation_sample = [valid_input_sample['title']['token'], valid_input_sample['title']['segment'], 
                   valid_input_sample['description']['token'], valid_input_sample['description']['segment'],
                   valid_input_sample['info'], valid_sim]

# Categorical columns
number_of_columns_info = valid_input_sample['info'].shape[1]
# Max sequence title
MAX_SEQUENCE_LENGTH_T = valid_input_sample['title']['token'].shape[1]
MAX_SEQUENCE_LENGTH_D = valid_input_sample['description']['token'].shape[1]

CPU times: user 65.4 ms, sys: 29 µs, total: 65.4 ms
Wall time: 65 ms


In [25]:
valid_input_sample['title']['token'].shape, \
valid_input_sample['description']['token'].shape, \
valid_input_sample['title']['segment'].shape, \
valid_input_sample['description']['segment'].shape, \
valid_input_sample['info'].shape, valid_sim.shape

((128, 20), (128, 20), (128, 1682), (128,))

### Validar entrada

In [ ]:
# %%time 

#baseline.display_batch(baseline.train_data, baseline.dup_sets_train, bug_train_ids, 5)

In [ ]:
"Test ", len(baseline.test_data)

## Experiment

## Propose

https://github.com/tqtg/DuplicateBugFinder

In [27]:
from keras.initializers import RandomUniform, RandomNormal, Ones

### BERT

https://github.com/CyberZHG/keras-bert

In [28]:
from keras_bert import load_trained_model_from_checkpoint
from keras_bert import compile_model, get_model
from keras.layers import GlobalAveragePooling1D

def bert_model(MAX_SEQUENCE_LENGTH, name):
    layer_num = 8
#     model = load_trained_model_from_checkpoint(
#             config_path,
#             model_path,
#             training=True,
#             trainable=True,
#             seq_len=MAX_SEQUENCE_LENGTH,
#     )
    model = load_trained_model_from_checkpoint(
        config_path,
        model_path,
        training=True,
        use_adapter=True,
        seq_len=MAX_SEQUENCE_LENGTH,
        trainable=['Encoder-{}-MultiHeadSelfAttention-Adapter'.format(i + 1) for i in range(12-layer_num, 13)] +
        ['Encoder-{}-FeedForward-Adapter'.format(i + 1) for i in range(12-layer_num, 13)] +
        ['Encoder-{}-MultiHeadSelfAttention-Norm'.format(i + 1) for i in range(12-layer_num, 13)] +
        ['Encoder-{}-FeedForward-Norm'.format(i + 1) for i in range(layer_num)],
    )
#     model = get_model(
#         token_num=len(token_dict),
#         head_num=10,
#         transformer_num=layer_num,
#         embed_dim=100,
#         feed_forward_dim=100,
#         seq_len=MAX_SEQUENCE_LENGTH,
#         pos_num=MAX_SEQUENCE_LENGTH,
#         dropout_rate=0.05,
#     )
    compile_model(model)
    inputs = model.inputs[:2]
    outputs = model.get_layer('Encoder-{}-FeedForward-Norm'.format(layer_num)).output
    #outputs = model.get_layer('Extract').output
    outputs = GlobalAveragePooling1D()(outputs)
    outputs = Dense(300, activation='tanh')(outputs)
    
    model = Model(inputs, outputs, name='FeatureBERTGenerationModel{}'.format(name))
    
    return model

### MLP

In [29]:
def mlp_model(input_size):
    info_input = Input(shape=(input_size, ), name='Feature_BugInput')
    input_size = 300
    
    for units in [64, 32]:
        layer = Dense(units, activation='tanh', kernel_initializer='random_uniform')(info_input)
    
    layer = Dense(input_size, activation='tanh')(info_input)
    
    mlp_feature_model = Model(inputs=[info_input], outputs=[layer], name = 'FeatureMlpGenerationModel')
    
    return mlp_feature_model

### Siamese model

In [30]:
def pairwise_distance(feature, squared=False):
    """Computes the pairwise distance matrix with numerical stability.

    output[i, j] = || feature[i, :] - feature[j, :] ||_2

    Args:
      feature: 2-D Tensor of size [number of data, feature dimension].
      squared: Boolean, whether or not to square the pairwise distances.

    Returns:
      pairwise_distances: 2-D Tensor of size [number of data, number of data].
    """
    pairwise_distances_squared = math_ops.add(
        math_ops.reduce_sum(math_ops.square(feature), axis=[1], keepdims=True),
        math_ops.reduce_sum(
            math_ops.square(array_ops.transpose(feature)),
            axis=[0],
            keepdims=True)) - 2.0 * math_ops.matmul(feature,
                                                    array_ops.transpose(feature))

    # Deal with numerical inaccuracies. Set small negatives to zero.
    pairwise_distances_squared = math_ops.maximum(pairwise_distances_squared, 0.0)
    # Get the mask where the zero distances are at.
    error_mask = math_ops.less_equal(pairwise_distances_squared, 0.0)

    # Optionally take the sqrt.
    if squared:
        pairwise_distances = pairwise_distances_squared
    else:
        pairwise_distances = math_ops.sqrt(
            pairwise_distances_squared + math_ops.to_float(error_mask) * 1e-16)

    # Undo conditionally adding 1e-16.
    pairwise_distances = math_ops.multiply(
        pairwise_distances, math_ops.to_float(math_ops.logical_not(error_mask)))

    num_data = array_ops.shape(feature)[0]
    # Explicitly set diagonals to zero.
    mask_offdiagonals = array_ops.ones_like(pairwise_distances) - array_ops.diag(
        array_ops.ones([num_data]))
    pairwise_distances = math_ops.multiply(pairwise_distances, mask_offdiagonals)
    return pairwise_distances

def masked_maximum(data, mask, dim=1):
    """Computes the axis wise maximum over chosen elements.

    Args:
      data: 2-D float `Tensor` of size [n, m].
      mask: 2-D Boolean `Tensor` of size [n, m].
      dim: The dimension over which to compute the maximum.

    Returns:
      masked_maximums: N-D `Tensor`.
        The maximized dimension is of size 1 after the operation.
    """
    axis_minimums = math_ops.reduce_min(data, dim, keepdims=True)
    masked_maximums = math_ops.reduce_max(
        math_ops.multiply(data - axis_minimums, mask), dim,
        keepdims=True) + axis_minimums
    return masked_maximums

def masked_minimum(data, mask, dim=1):
    """Computes the axis wise minimum over chosen elements.

    Args:
      data: 2-D float `Tensor` of size [n, m].
      mask: 2-D Boolean `Tensor` of size [n, m].
      dim: The dimension over which to compute the minimum.

    Returns:
      masked_minimums: N-D `Tensor`.
        The minimized dimension is of size 1 after the operation.
    """
    axis_maximums = math_ops.reduce_max(data, dim, keepdims=True)
    masked_minimums = math_ops.reduce_min(
        math_ops.multiply(data - axis_maximums, mask), dim,
        keepdims=True) + axis_maximums
    return masked_minimums

In [ ]:
## required for semi-hard triplet loss:
import tensorflow as tf
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.framework import dtypes

def triplet_loss(vects):
    margin = 1.
    labels = vects[:, :1]
 
    labels = tf.cast(labels, dtype='int32')

    embeddings = tf.cast(vects[:, 1:], dtype='float32')

    ### Code from Tensorflow function [tf.contrib.losses.metric_learning.triplet_semihard_loss] starts here:
    
    # Reshape [batch_size] label tensor to a [batch_size, 1] label tensor.
    # lshape=array_ops.shape(labels)
    # assert lshape.shape == 1
    # labels = array_ops.reshape(labels, [lshape[0], 1])

    # Build pairwise squared distance matrix.
    pdist_matrix = pairwise_distance(embeddings, squared=True)
    # Build pairwise binary adjacency matrix.
    adjacency = math_ops.equal(labels, array_ops.transpose(labels))
    # Invert so we can select negatives only.
    adjacency_not = math_ops.logical_not(adjacency)

    # global batch_size  
    batch_size = array_ops.size(labels) # was 'array_ops.size(labels)'

    # Compute the mask.
    pdist_matrix_tile = array_ops.tile(pdist_matrix, [batch_size, 1])
    mask = math_ops.logical_and(
        array_ops.tile(adjacency_not, [batch_size, 1]),
        math_ops.greater(
            pdist_matrix_tile, array_ops.reshape(
                array_ops.transpose(pdist_matrix), [-1, 1])))
    mask_final = array_ops.reshape(
        math_ops.greater(
            math_ops.reduce_sum(
                math_ops.cast(mask, dtype=dtypes.float32), 1, keepdims=True),
            0.0), [batch_size, batch_size])
    mask_final = array_ops.transpose(mask_final)

    adjacency_not = math_ops.cast(adjacency_not, dtype=dtypes.float32)
    mask = math_ops.cast(mask, dtype=dtypes.float32)

    # negatives_outside: smallest D_an where D_an > D_ap.
    negatives_outside = array_ops.reshape(
        masked_minimum(pdist_matrix_tile, mask), [batch_size, batch_size])
    negatives_outside = array_ops.transpose(negatives_outside)

    # negatives_inside: largest D_an.
    negatives_inside = array_ops.tile(
        masked_maximum(pdist_matrix, adjacency_not), [1, batch_size])
    semi_hard_negatives = array_ops.where(
        mask_final, negatives_outside, negatives_inside)

    loss_mat = math_ops.add(margin, pdist_matrix - semi_hard_negatives)

    mask_positives = math_ops.cast(
        adjacency, dtype=dtypes.float32) - array_ops.diag(
        array_ops.ones([batch_size]))

    # In lifted-struct, the authors multiply 0.5 for upper triangular
    #   in semihard, they take all positive pairs except the diagonal.
    num_positives = math_ops.reduce_sum(mask_positives)

    semi_hard_triplet_loss_distance = math_ops.truediv(
        math_ops.reduce_sum(
            math_ops.maximum(
                math_ops.multiply(loss_mat, mask_positives), 0.0)),
        num_positives,
        name='triplet_semihard_loss')
    
    ### Code from Tensorflow function semi-hard triplet loss ENDS here.
    return semi_hard_triplet_loss_distance

def quintet_loss(inputs):
    margin = 1.
    labels = inputs[:, :1]
 
    labels = tf.cast(labels, dtype='int32')

    embeddings = inputs[:, 1:]

    # Build pairwise squared distance matrix.
    pdist_matrix = pairwise_distance(embeddings, squared=True)
    # Build pairwise binary adjacency matrix.
    adjacency = math_ops.equal(labels, array_ops.transpose(labels))
    # Invert so we can select negatives only.
    adjacency_not = math_ops.logical_not(adjacency)

    # global batch_size  
    batch_size = array_ops.size(labels) # was 'array_ops.size(labels)'

    adjacency_not = math_ops.cast(adjacency_not, dtype=dtypes.float32)

    mask_positives = math_ops.cast(
        adjacency, dtype=dtypes.float32) - array_ops.diag(array_ops.ones([batch_size]))

    # In lifted-struct, the authors multiply 0.5 for upper triangular
    #   in semihard, they take all positive pairs except the diagonal.
    num_positives = math_ops.reduce_sum(mask_positives)
    
    mask_negatives = adjacency_not
    
    # Include the anchor to positives
    mask_positives_centroids = math_ops.cast(adjacency, dtype=dtypes.float32)

#     return mask_positives

   # pos 
    embed_pos = tf.matmul(mask_positives_centroids, embeddings)
    num_of_pos = tf.reduce_sum(mask_positives_centroids, axis=1, keepdims=True)
    centroid_embed_pos = tf.math.xdivy(embed_pos, num_of_pos)
    labels_pos = tf.cast(labels, dtype=dtypes.float32)
    # negs
    embed_neg = tf.matmul(mask_negatives, embeddings)
    num_of_neg = tf.reduce_sum(mask_negatives, axis=1, keepdims=True)
    centroid_embed_neg = tf.math.xdivy(embed_neg, num_of_neg)

#     return mask_positives_centroids
    i = tf.constant(0)
    batch_centroid_matrix = tf.Variable([])
    batch_centroid_matrix_neg = tf.Variable([])
    batch_centroid_matrix_all = tf.Variable([])
    def iter_centroids(i, batch_centroid_matrix, batch_centroid_matrix_neg, batch_centroid_matrix_all):
        # centroid pos
        mask_positives_batch = tf.reshape(tf.gather(mask_positives, i), (-1, 1))
        centroid_pos = tf.gather(centroid_embed_pos, i)
        
        centroid_embed = tf.repeat([centroid_pos], repeats=[batch_size], axis=0)
        new_batch_centroid_pos = mask_positives_batch * centroid_embed
        new_batch_embeddings = tf.cast(tf.logical_not(tf.cast(mask_positives_batch, 'bool')), 'float32') * embeddings 
        new_batch = tf.reduce_sum([new_batch_centroid_pos, new_batch_embeddings], axis=0, keepdims=True)[0]
        
        vects_new_batch = tf.concat([labels_pos, new_batch], axis=1)
        TL_new_batch = triplet_loss(vects_new_batch)
        batch_centroid_matrix = tf.concat([batch_centroid_matrix, [TL_new_batch]], axis=0) 
        
        # centroid neg
        centroid_neg = tf.gather(centroid_embed_neg, i)
        mask_negatives_batch = tf.reshape(tf.gather(mask_negatives, i), (-1, 1))
        
        centroid_embed = tf.repeat([centroid_neg], repeats=[batch_size], axis=0)
        new_batch_centroid_neg = mask_negatives_batch * centroid_embed
        new_batch_embeddings = tf.cast(tf.logical_not(tf.cast(mask_negatives_batch, 'bool')), 'float32') * embeddings 
        new_batch = tf.reduce_sum([new_batch_centroid_neg, new_batch_embeddings], axis=0, keepdims=True)[0]
        
        vects_new_batch = tf.concat([labels_pos, new_batch], axis=1)
        TL_new_batch = triplet_loss(vects_new_batch)
        batch_centroid_matrix_neg = tf.concat([batch_centroid_matrix_neg, [TL_new_batch]], axis=0) 
        
        # centroid pos and neg
        new_batch_centroids = tf.reduce_sum([new_batch_centroid_pos, new_batch_centroid_neg], axis=0, keepdims=True)[0]
        vects_new_batch_centroids = tf.concat([labels_pos, new_batch_centroids], axis=1)
        TL_new_batch = triplet_loss(vects_new_batch_centroids)
        batch_centroid_matrix_all = tf.concat([batch_centroid_matrix_all, [TL_new_batch]], axis=0) 
        
        return [tf.add(i, 1), batch_centroid_matrix, batch_centroid_matrix_neg, batch_centroid_matrix_all]
    _, batch_centroid_matrix, batch_centroid_matrix_neg, batch_centroid_matrix_all = tf.while_loop(lambda i, a, b, c: i<batch_size, 
                                        iter_centroids, 
                                        [i, batch_centroid_matrix, batch_centroid_matrix_neg, batch_centroid_matrix_all],
                                       shape_invariants=[i.get_shape(),
                                                   tf.TensorShape([None]), tf.TensorShape([None]), tf.TensorShape([None])])

    TL_anchor_w = 1.0 # tf.random_uniform_initializer(minval=0.0, maxval=1.)(shape=[1])[0]
    TL_pos_w = 1.0 # tf.random_uniform_initializer(minval=0.0, maxval=1.)(shape=[1])[0]
    TL_neg_w = 1.0 # tf.random_uniform_initializer(minval=0.0, maxval=1.)(shape=[1])[0]
    TL_centroid_w = 1.0 # tf.random_uniform_initializer(minval=0.0, maxval=1.)(shape=[1])[0]
    
#     tl_weights = tf.truediv(num_of_pos, tf.reduce_max(num_of_pos))
#     tl_w = tl_weights # tf.random_uniform_initializer(minval=0.0, maxval=tl_weights)(shape=[1])
#     TL_pos_weighted = tf.reshape(batch_centroid_matrix, (-1, 1)) * tl_w
#     TL_pos = tf.truediv(tf.reduce_sum(TL_pos_weighted), tf.reduce_sum(tl_w))

    TL = triplet_loss(inputs)
    TL_pos = tf.reduce_mean(batch_centroid_matrix)
    TL_neg = tf.reduce_mean(batch_centroid_matrix_neg) #triplet_loss(vects_neg)
    TL_centroid = tf.reduce_mean(batch_centroid_matrix_all) # triplet_loss(vects_centroids)

    sum_of_median = tf.reduce_sum([TL * TL_anchor_w, TL_pos * TL_pos_w, TL_neg * TL_neg_w, TL_centroid * TL_centroid_w]) # 
    sum_of_weigths = TL_anchor_w + TL_pos_w + TL_neg_w + TL_centroid_w
    weigthed_median = tf.truediv(sum_of_median, sum_of_weigths)    
    return tf.cast([weigthed_median, TL_anchor_w, TL_pos_w, TL_neg_w, 
                    TL_centroid_w, TL, TL_pos, TL_neg, TL_centroid], 
                   dtype=dtypes.float32)


def custom_loss(y_true, y_pred):
    return tf.reduce_mean(y_pred[0])

def TL_w_anchor(y_true, y_pred):
    return tf.reduce_mean(y_pred[1])
def TL_w_pos(y_true, y_pred):
    return tf.reduce_mean(y_pred[2])
def TL_w_neg(y_true, y_pred):
    return tf.reduce_mean(y_pred[3])
def TL_w_centroid(y_true, y_pred):
    return tf.reduce_mean(y_pred[4])
def TL(y_true, y_pred):
    return tf.reduce_mean(y_pred[5])
def TL_pos(y_true, y_pred):
    return tf.reduce_mean(y_pred[6])
def TL_neg(y_true, y_pred):
    return tf.reduce_mean(y_pred[7])
def TL_centroid(y_true, y_pred):
    return tf.reduce_mean(y_pred[8])

### Propose

In [31]:
from keras.layers import concatenate, Add, Lambda, merge, Average, Maximum
from keras.optimizers import Adam, Nadam

def siamese_model(title_feature_model, desc_feature_model, categorical_feature_model, sequence_length_info, 
                  sequence_length_t, sequence_length_d, name):
  
    # Title
    bug_t_token = Input(shape = (sequence_length_t, ), name = 'title_token_{}'.format(name))
    bug_t_segment = Input(shape = (sequence_length_t, ), name = 'title_segment_{}'.format(name))
    # Description
    bug_d_token = Input(shape = (sequence_length_d, ), name = 'desc_token_{}'.format(name))
    bug_d_segment = Input(shape = (sequence_length_d, ), name = 'desc_segment_{}'.format(name))
    # Categorical
    bug_i = Input(shape = (sequence_length_info, ), name = 'info_{}'.format(name))
    
    bug_t_feat = title_feature_model([bug_t_token, bug_t_segment])
    bug_d_feat = desc_feature_model([bug_d_token, bug_d_segment])
    bug_i_feat = categorical_feature_model(bug_i)
    
    #bug_feature_output = Add(name = 'merge_features_{}'.format(name))([bug_i_feat, bug_t_feat, bug_d_feat])
    bug_feature_output = concatenate([bug_i_feat, bug_t_feat, bug_d_feat], name = 'merge_features_{}'.format(name))
    
    bug_feature_model = Model(inputs=[bug_t_token, bug_t_segment, bug_d_token, bug_d_segment, bug_i], outputs=[bug_feature_output], name = 'merge_features_{}'.format(name))
    
    return bug_feature_model

In [46]:
def max_margin_objective(encoded_anchor, decay_lr=1):
    
    input_labels = Input(shape=(1,), name='input_label')    # input layer for labels
    inputs = np.concatenate([encoded_anchor.input, [input_labels]], -1).tolist()
    
    encoded_anchor = encoded_anchor.output
    
    output = concatenate([input_labels, encoded_anchor])  # concatenating the labels + embeddings
    
    output = Lambda(quintet_loss, name='quintet_loss')(output)
    
    similarity_model = Model(inputs = inputs, outputs = output, name = 'Similarity_Model')

    #optimizer = Nadam(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=K.epsilon(), schedule_decay=0.01)
    # optimizer = Adam(lr=1e-3 * decay_lr, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)

    # setup the optimization process 
    similarity_model.compile(optimizer='adam', loss=custom_loss, metrics=[TL_w_anchor, TL_w_pos, TL_w_neg, TL_w_centroid,
                                                                         TL, TL_pos, TL_neg, TL_centroid]) 
    # metrics=[pos_distance, neg_distance, custom_margin_loss]

    return similarity_model

In [ ]:
import _pickle as pickle
def save_loss(result):
    with open(os.path.join(DIR,'{}_log.pkl'.format(METHOD)), 'wb') as f:
        pickle.dump(result, f)
    print("=> result saved!")

In [47]:
%%time
import keras

# Inspired on https://'pastebin.com/TaGFdcBA
# TODO: https://stackoverflow.com/questions/49941903/keras-compute-cosine-distance-between-two-flattened-outputs
keras.backend.clear_session()

# Clear GPU memory
# from numba import cuda
# cuda.select_device(0)
# cuda.close()

# Feature models
'''
    cnn_model
    lstm_model
    mlp_model
'''
title_feature_model = bert_model(MAX_SEQUENCE_LENGTH_T, 'Title')
desc_feature_model = bert_model(MAX_SEQUENCE_LENGTH_D, 'Description')
categorical_feature_model = mlp_model(number_of_columns_info)

# Similarity model
encoded_anchor = siamese_model(title_feature_model, desc_feature_model, categorical_feature_model, 
                                     number_of_columns_info, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D, 'in')

similarity_model = max_margin_objective(encoded_anchor, decay_lr=1)

# cnn_feature_model.summary()
# lstm_feature_model.summary()
similarity_model.summary()

'''
    Experiment
'''
result = { 'train' : [], 'test' : [] }
limit_train = int(epochs * freeze_train) # 10% de 1000 , 100 epocas
print("Total of ", limit_train)
for epoch in range(limit_train):
    batch_triplet_train, \
        train_input_sample, train_sim = batch_iterator(baseline, retrieval, encoded_anchor, baseline.train_data, 
                                                       baseline.dup_sets_train, bug_train_ids, 
                                                           batch_size, 1, issues_by_buckets, TRIPLET_HARD=False)
    train_batch = [train_input_sample['title']['token'], train_input_sample['title']['segment'], 
                   train_input_sample['description']['token'], train_input_sample['description']['segment'],
                   train_input_sample['info'], train_sim]
    
#     if epoch == 10:
#         similarity_model = max_margin_objective(encoded_anchor, encoded_positive, encoded_negative, decay_lr=0.1)
    
    h = similarity_model.train_on_batch(x=train_batch, y=train_sim)
    h_validation = similarity_model.test_on_batch(x=validation_sample, y=valid_sim)
    
    # save results
    result['train'].append(h)
    result['test'].append(h_validation)
    
    if( (epoch+1) % 10 == 0 or (epoch+1 == limit_train) ):
        save_loss(result)
    
    if (epoch+1 == limit_train): #(epoch > 1 and epoch % 10 == 0) or (epoch+1 == epochs):
        recall, exported_rank, debug = experiment.evaluate_validation_test(retrieval, verbose, encoded_anchor, issues_by_buckets, 
                                                               bug_train_ids, method='bert')
        print(("Epoch: {} Loss: {:.2f}, Loss_test: {:.2f}\n" +
               "TL_w: {:.2f}, TL_pos_w: {:.2f}, TL_neg_w: {:.2f}, TL_centroid_w: {:.2f}\n" + 
                "TL: {:.2f}, TL_pos: {:.2f}, TL_neg: {:.2f}, TL_centroid: {:.2f}, " +
              "recall@25: {:.2f}").format(epoch+1, h[0], h_validation[0], h[1], h[2], h[3], 
                                          h[4], h[5], h[6], h[7], h[8], recall))
    else:
        print(("Epoch: {} Loss: {:.2f}, Loss_test: {:.2f}\n" +
               "TL_w: {:.2f}, TL_pos_w: {:.2f}, TL_neg_w: {:.2f}, TL_centroid_w: {:.2f}\n" + 
              "TL: {:.2f}, TL_pos: {:.2f}, TL_neg: {:.2f}, TL_centroid: {:.2f}").format(
            epoch+1, h[0], h_validation[0], h[1], h[2], h[3], h[4], h[5], h[6], h[7], h[8]))
    loss = h[0]
    
    if loss < best_loss:
        best_loss = loss
        best_epoch = epoch+1

# experiment.save_model(similarity_model, SAVE_PATH.replace('@number_of_epochs@', str(epochs)))
# experiment.save_model(encoded_anchor, SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs)), verbose=1)
print('Best_epoch={}, Best_loss={:.2f}, Recall@25={:.2f}'.format(best_epoch, best_loss, recall))

Batch size  64
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
info_in (InputLayer)            (None, 1682)         0                                            
__________________________________________________________________________________________________
title_token_in (InputLayer)     (None, 20)           0                                            
__________________________________________________________________________________________________
title_segment_in (InputLayer)   (None, 20)           0                                            
__________________________________________________________________________________________________
desc_token_in (InputLayer)      (None, 20)           0                                            
______________________________________________________________________________________________

Epoch: 1 Loss: 0.38, TL: 1.02, TL_A: 0.97, TL_P: 0.96, TL_N: 0.97
Epoch: 2 Loss: 0.38, TL: 1.02, TL_A: 0.97, TL_P: 0.96, TL_N: 0.97
Epoch: 3 Loss: 0.38, TL: 1.02, TL_A: 0.96, TL_P: 0.95, TL_N: 0.97
Epoch: 4 Loss: 0.38, TL: 1.02, TL_A: 0.96, TL_P: 0.95, TL_N: 0.97
Epoch: 5 Loss: 0.38, TL: 1.01, TL_A: 0.96, TL_P: 0.95, TL_N: 0.96
Epoch: 6 Loss: 0.38, TL: 1.01, TL_A: 0.96, TL_P: 0.94, TL_N: 0.96
Epoch: 7 Loss: 0.37, TL: 1.02, TL_A: 0.97, TL_P: 0.95, TL_N: 0.96
Epoch: 8 Loss: 0.37, TL: 1.02, TL_A: 0.96, TL_P: 0.95, TL_N: 0.97
Epoch: 9 Loss: 0.37, TL: 1.02, TL_A: 0.96, TL_P: 0.95, TL_N: 0.97
Epoch: 10 Loss: 0.37, TL: 1.01, TL_A: 0.96, TL_P: 0.95, TL_N: 0.96
Epoch: 11 Loss: 0.37, TL: 1.02, TL_A: 0.96, TL_P: 0.95, TL_N: 0.97
Epoch: 12 Loss: 0.37, TL: 1.02, TL_A: 0.96, TL_P: 0.95, TL_N: 0.96
Epoch: 13 Loss: 0.37, TL: 1.02, TL_A: 0.96, TL_P: 0.95, TL_N: 0.96
Epoch: 14 Loss: 0.37, TL: 1.02, TL_A: 0.96, TL_P: 0.95, TL_N: 0.96
Epoch: 15 Loss: 0.37, TL: 1.03, TL_A: 0.95, TL_P: 0.94, TL_N: 0.96
Epoc

Epoch: 124 Loss: 0.22, TL: 1.27, TL_A: 0.96, TL_P: 0.96, TL_N: 0.98
Epoch: 125 Loss: 0.23, TL: 1.35, TL_A: 0.97, TL_P: 0.84, TL_N: 0.99
Epoch: 126 Loss: 0.24, TL: 1.30, TL_A: 0.97, TL_P: 0.84, TL_N: 0.99
Epoch: 127 Loss: 0.23, TL: 1.32, TL_A: 1.00, TL_P: 0.82, TL_N: 0.97
Epoch: 128 Loss: 0.20, TL: 1.28, TL_A: 1.08, TL_P: 0.85, TL_N: 0.88
Epoch: 129 Loss: 0.22, TL: 1.38, TL_A: 1.00, TL_P: 0.80, TL_N: 0.96
Epoch: 130 Loss: 0.21, TL: 1.41, TL_A: 1.00, TL_P: 0.84, TL_N: 0.97
Epoch: 131 Loss: 0.23, TL: 1.30, TL_A: 0.99, TL_P: 0.78, TL_N: 0.95
Epoch: 132 Loss: 0.20, TL: 1.40, TL_A: 1.04, TL_P: 0.84, TL_N: 0.92
Epoch: 133 Loss: 0.27, TL: 1.30, TL_A: 0.84, TL_P: 0.82, TL_N: 1.11
Epoch: 134 Loss: 0.23, TL: 1.31, TL_A: 1.00, TL_P: 0.85, TL_N: 0.98
Epoch: 135 Loss: 0.25, TL: 1.30, TL_A: 0.93, TL_P: 0.83, TL_N: 1.05
Epoch: 136 Loss: 0.24, TL: 1.26, TL_A: 1.01, TL_P: 0.81, TL_N: 0.99
Epoch: 137 Loss: 0.24, TL: 1.32, TL_A: 0.90, TL_P: 0.84, TL_N: 1.04
Epoch: 138 Loss: 0.24, TL: 1.31, TL_A: 0.95, TL_

Epoch: 245 Loss: 0.13, TL: 1.11, TL_A: 0.99, TL_P: 0.98, TL_N: 1.00
Epoch: 246 Loss: 0.13, TL: 1.10, TL_A: 0.99, TL_P: 0.97, TL_N: 0.99
Epoch: 247 Loss: 0.14, TL: 1.01, TL_A: 1.00, TL_P: 0.99, TL_N: 0.99
Epoch: 248 Loss: 0.13, TL: 1.08, TL_A: 0.99, TL_P: 0.98, TL_N: 0.99
Epoch: 249 Loss: 0.13, TL: 1.08, TL_A: 1.02, TL_P: 0.96, TL_N: 0.98
Epoch: 250 Loss: 0.13, TL: 1.08, TL_A: 0.98, TL_P: 1.00, TL_N: 1.00
Epoch: 251 Loss: 0.13, TL: 1.06, TL_A: 0.98, TL_P: 0.97, TL_N: 0.99
Epoch: 252 Loss: 0.13, TL: 1.06, TL_A: 1.00, TL_P: 0.98, TL_N: 0.99
Epoch: 253 Loss: 0.12, TL: 1.09, TL_A: 0.98, TL_P: 0.99, TL_N: 1.00
Epoch: 254 Loss: 0.12, TL: 1.07, TL_A: 0.99, TL_P: 0.99, TL_N: 0.99
Epoch: 255 Loss: 0.12, TL: 1.10, TL_A: 0.99, TL_P: 0.99, TL_N: 0.99
Epoch: 256 Loss: 0.11, TL: 1.08, TL_A: 1.03, TL_P: 0.98, TL_N: 0.97
Epoch: 257 Loss: 0.10, TL: 1.17, TL_A: 1.03, TL_P: 0.93, TL_N: 0.95
Epoch: 258 Loss: 0.11, TL: 1.14, TL_A: 1.00, TL_P: 0.98, TL_N: 0.99
Epoch: 259 Loss: 0.12, TL: 1.08, TL_A: 1.00, TL_

Epoch: 366 Loss: -0.02, TL: 1.24, TL_A: 0.95, TL_P: 0.97, TL_N: 0.98
Epoch: 367 Loss: -0.03, TL: 1.31, TL_A: 1.04, TL_P: 0.88, TL_N: 0.93
Epoch: 368 Loss: -0.01, TL: 1.31, TL_A: 1.04, TL_P: 0.78, TL_N: 0.94
Epoch: 369 Loss: 0.02, TL: 1.25, TL_A: 0.93, TL_P: 0.88, TL_N: 1.07
Epoch: 370 Loss: -0.01, TL: 1.30, TL_A: 1.07, TL_P: 0.80, TL_N: 0.93
Epoch: 371 Loss: -0.03, TL: 1.34, TL_A: 0.96, TL_P: 0.80, TL_N: 0.93
Epoch: 372 Loss: -0.05, TL: 1.43, TL_A: 1.08, TL_P: 0.76, TL_N: 0.86
Epoch: 373 Loss: 0.01, TL: 1.29, TL_A: 0.93, TL_P: 0.77, TL_N: 1.02
Epoch: 374 Loss: 0.01, TL: 1.32, TL_A: 1.00, TL_P: 0.76, TL_N: 1.00
Epoch: 375 Loss: -0.06, TL: 1.38, TL_A: 1.11, TL_P: 0.78, TL_N: 0.83
Epoch: 376 Loss: -0.03, TL: 1.35, TL_A: 1.10, TL_P: 0.70, TL_N: 0.88
Epoch: 377 Loss: -0.03, TL: 1.28, TL_A: 0.97, TL_P: 0.97, TL_N: 0.99
Epoch: 378 Loss: -0.00, TL: 1.22, TL_A: 1.03, TL_P: 0.77, TL_N: 0.93
Epoch: 379 Loss: -0.00, TL: 1.25, TL_A: 0.90, TL_P: 0.91, TL_N: 1.06
Epoch: 380 Loss: -0.03, TL: 1.23, TL_

Epoch: 485 Loss: -0.13, TL: 1.21, TL_A: 1.00, TL_P: 0.89, TL_N: 0.95
Epoch: 486 Loss: -0.11, TL: 1.38, TL_A: 0.94, TL_P: 0.80, TL_N: 1.05
Epoch: 487 Loss: -0.15, TL: 1.38, TL_A: 1.05, TL_P: 0.86, TL_N: 0.94
Epoch: 488 Loss: -0.12, TL: 1.27, TL_A: 1.03, TL_P: 0.77, TL_N: 0.96
Epoch: 489 Loss: -0.13, TL: 1.36, TL_A: 0.98, TL_P: 0.80, TL_N: 0.96
Epoch: 490 Loss: -0.12, TL: 1.44, TL_A: 0.91, TL_P: 0.73, TL_N: 1.03
Epoch: 491 Loss: -0.12, TL: 1.30, TL_A: 0.95, TL_P: 0.89, TL_N: 1.03
Epoch: 492 Loss: -0.06, TL: 1.31, TL_A: 0.83, TL_P: 0.72, TL_N: 1.16
Epoch: 493 Loss: -0.12, TL: 1.36, TL_A: 0.97, TL_P: 0.78, TL_N: 1.01
Epoch: 494 Loss: -0.14, TL: 1.37, TL_A: 0.96, TL_P: 0.90, TL_N: 1.03
Epoch: 495 Loss: -0.16, TL: 1.42, TL_A: 1.03, TL_P: 0.85, TL_N: 0.94
Epoch: 496 Loss: -0.10, TL: 1.24, TL_A: 0.94, TL_P: 0.74, TL_N: 1.00
Epoch: 497 Loss: -0.17, TL: 1.35, TL_A: 1.05, TL_P: 0.87, TL_N: 0.91
Epoch: 498 Loss: -0.13, TL: 1.40, TL_A: 0.97, TL_P: 0.75, TL_N: 1.00
Epoch: 499 Loss: -0.12, TL: 1.25, 

Epoch: 604 Loss: -0.21, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 605 Loss: -0.21, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 606 Loss: -0.22, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 607 Loss: -0.22, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 608 Loss: -0.22, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 609 Loss: -0.22, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 610 Loss: -0.22, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 611 Loss: -0.22, TL: 1.00, TL_A: 1.00, TL_P: 0.99, TL_N: 1.00
Epoch: 612 Loss: -0.22, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 613 Loss: -0.22, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 614 Loss: -0.22, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 615 Loss: -0.23, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 616 Loss: -0.23, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 617 Loss: -0.23, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 618 Loss: -0.23, TL: 1.00, 

Epoch: 723 Loss: -0.36, TL: 1.19, TL_A: 0.99, TL_P: 0.97, TL_N: 1.00
Epoch: 724 Loss: -0.36, TL: 1.23, TL_A: 1.03, TL_P: 0.91, TL_N: 0.97
Epoch: 725 Loss: -0.37, TL: 1.27, TL_A: 1.00, TL_P: 0.92, TL_N: 1.00
Epoch: 726 Loss: -0.36, TL: 1.26, TL_A: 0.97, TL_P: 0.95, TL_N: 1.04
Epoch: 727 Loss: -0.35, TL: 1.14, TL_A: 1.00, TL_P: 0.89, TL_N: 0.98
Epoch: 728 Loss: -0.36, TL: 1.21, TL_A: 1.02, TL_P: 0.89, TL_N: 0.97
Epoch: 729 Loss: -0.36, TL: 1.26, TL_A: 0.96, TL_P: 0.93, TL_N: 1.04
Epoch: 730 Loss: -0.37, TL: 1.22, TL_A: 1.02, TL_P: 0.94, TL_N: 1.00
Epoch: 731 Loss: -0.36, TL: 1.17, TL_A: 1.00, TL_P: 0.93, TL_N: 0.99
Epoch: 732 Loss: -0.35, TL: 1.09, TL_A: 0.99, TL_P: 0.93, TL_N: 0.98
Epoch: 733 Loss: -0.37, TL: 1.20, TL_A: 1.00, TL_P: 0.95, TL_N: 1.01
Epoch: 734 Loss: -0.38, TL: 1.27, TL_A: 1.01, TL_P: 0.93, TL_N: 1.00
Epoch: 735 Loss: -0.36, TL: 1.25, TL_A: 0.99, TL_P: 0.89, TL_N: 1.00
Epoch: 736 Loss: -0.34, TL: 1.14, TL_A: 0.95, TL_P: 0.88, TL_N: 1.03
Epoch: 737 Loss: -0.39, TL: 1.31, 

Epoch: 842 Loss: -0.45, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 843 Loss: -0.45, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 844 Loss: -0.46, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 845 Loss: -0.46, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 846 Loss: -0.46, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 847 Loss: -0.46, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 848 Loss: -0.46, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 849 Loss: -0.46, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 850 Loss: -0.46, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 851 Loss: -0.46, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 852 Loss: -0.46, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 853 Loss: -0.46, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 854 Loss: -0.47, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 855 Loss: -0.47, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 856 Loss: -0.47, TL: 1.00, 

Epoch: 961 Loss: -0.57, TL: 1.01, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 962 Loss: -0.57, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 963 Loss: -0.57, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 964 Loss: -0.58, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 965 Loss: -0.58, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 966 Loss: -0.58, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 967 Loss: -0.58, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 968 Loss: -0.58, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 969 Loss: -0.58, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 970 Loss: -0.58, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 971 Loss: -0.58, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 972 Loss: -0.58, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 973 Loss: -0.58, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 974 Loss: -0.59, TL: 1.00, TL_A: 1.00, TL_P: 1.00, TL_N: 1.00
Epoch: 975 Loss: -0.59, TL: 1.00, 

In [ ]:
# Domain to use
METHOD = 'deepQL_weights_{}'.format(limit_train)
SAVE_PATH = '{}_preprocessing_{}_feature@number_of_epochs@epochs_64batch({})'.format(PREPROCESSING, METHOD, DOMAIN)
SAVE_PATH_FEATURE = '{}_preprocessing_{}_feature_@number_of_epochs@epochs_64batch({})'.format(PREPROCESSING, METHOD, DOMAIN)

In [ ]:
EXPORT_RANK_PATH = os.path.join(DIR, 'exported_rank_{}.txt'.format(METHOD))
EXPORT_RANK_PATH

In [ ]:
with open(EXPORT_RANK_PATH, 'w') as file_out:
    for row in exported_rank:
        file_out.write(row + "\n")

In [ ]:
experiment.save_model(similarity_model, SAVE_PATH.replace('@number_of_epochs@', str(limit_train)))
experiment.save_model(encoded_anchor, SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(limit_train)), verbose=1)

In [ ]:
len(result['train']), len(result['test'])

In [ ]:
model = similarity_model.get_layer('concatenate_1')
output = Lambda(quintet_loss, name='quintet_loss_unfreezed')(model.output)
inputs = similarity_model.inputs
model = Model(inputs = inputs, outputs = output, name = 'Similarity_Model')

# setup the optimization process 
model.compile(optimizer='adam', loss=custom_loss, metrics=[TL_w_anchor, TL_w_pos, TL_w_neg, TL_w_centroid,
                                                                     TL, TL_pos, TL_neg, TL_centroid])
model.summary()

In [ ]:
end_train = epochs - limit_train
for epoch in range(limit_train, end_train):
    batch_triplet_train, \
        train_input_sample, train_sim = batch_iterator(baseline, retrieval, model, baseline.train_data, 
                                                       baseline.dup_sets_train, bug_train_ids, 
                                                           batch_size, 1, issues_by_buckets, TRIPLET_HARD=False)
    train_batch = [train_input_sample['title']['token'], train_input_sample['title']['segment'], 
                   train_input_sample['description']['token'], train_input_sample['description']['segment'],
                   train_input_sample['info'], train_sim]
    

    h = model.train_on_batch(x=train_batch, y=train_sim)
    h_validation = model.test_on_batch(x=validation_sample, y=valid_sim)
    
    # save results
    result['train'].append(h)
    result['test'].append(h_validation)
    
    if( (epoch+1) % 10 == 0 or (epoch+1 == end_train )):
        save_loss(result)
    
    print(("Epoch: {} Loss: {:.2f}, Loss_test: {:.2f}\n" +
               "TL_w: {:.2f}, TL_pos_w: {:.2f}, TL_neg_w: {:.2f}, TL_centroid_w: {:.2f}\n" + 
              "TL: {:.2f}, TL_pos: {:.2f}, TL_neg: {:.2f}, TL_centroid: {:.2f}").format(
            epoch+1, h[0], h_validation[0], h[1], h[2], h[3], h[4], h[5], h[6], h[7], h[8]))

In [ ]:
len(result['train']), len(result['test'])

In [ ]:
encoded = model.get_layer('merge_features_in')
output = encoded.output
inputs = similarity_model.inputs[:-1]
encoded_anchor = Model(inputs = inputs, outputs = output, name = 'Similarity_Model')

In [ ]:
# Domain to use
METHOD = 'deepQL_weights_{}'.format(epochs)
SAVE_PATH = '{}_preprocessing_{}_feature@number_of_epochs@epochs_64batch({})'.format(PREPROCESSING, METHOD, DOMAIN)
SAVE_PATH_FEATURE = '{}_preprocessing_{}_feature_@number_of_epochs@epochs_64batch({})'.format(PREPROCESSING, METHOD, DOMAIN)

In [ ]:
SAVE_PATH.replace('@number_of_epochs@', str(epochs))

In [ ]:
experiment.save_model(model, SAVE_PATH.replace('@number_of_epochs@', str(epochs)))
experiment.save_model(encoded_anchor, SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs)), verbose=1)
"Model saved"

In [ ]:
recall, exported_rank, debug = experiment.evaluate_validation_test(retrieval, 1, encoded_anchor, issues_by_buckets, 
                                                               bug_train_ids, method='bert')
print(("Epoch: {} Loss: {:.2f}, Loss_test: {:.2f}\n" +
       "TL_w: {:.2f}, TL_pos_w: {:.2f}, TL_neg_w: {:.2f}, TL_centroid_w: {:.2f}\n" + 
        "TL: {:.2f}, TL_pos: {:.2f}, TL_neg: {:.2f}, TL_centroid: {:.2f}, " +
      "recall@25: {:.2f}").format(epoch+1, h[0], h_validation[0], h[1], h[2], h[3], h[4], h[5], h[6], h[7], h[8], recall))

In [ ]:
recall

In [ ]:
exported_rank[:20]

In [ ]:
print("Total of queries:", len(retrieval.test))

In [ ]:
SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs))

In [ ]:
encoded_anchor.summary()

In [ ]:
len(exported_rank)

In [ ]:
EXPORT_RANK_PATH = os.path.join(DIR, 'exported_rank_{}.txt'.format(METHOD))
EXPORT_RANK_PATH

In [ ]:
report = experiment.evaluation.evaluate(EXPORT_RANK_PATH)
report